In [4]:
import os

try:
    os.remove("metastore_db/db.lck")
    os.remove("metastore_db/dbex.lck")
except:
    pass


from pyspark.sql import SparkSession

def build_spark_session(app_name, memory='4g', executors=4):
    return SparkSession.builder\
                      .appName(app_name)\
                      .config('spark.executor.memory', memory)\
                      .config('spark.executor.instances', executors)\
                      .getOrCreate()

spark_session = build_spark_session(app_name='ok-google')

from pyspark.sql import functions as f

In [3]:
from pyspark.sql import Row
import datetime

l = [(datetime.date(2018,1,3), 'Ankit',25,"F"),
     (datetime.date(2018,2,3), 'Jalfaizy',22,"F"),
     (datetime.date(2018,1,5), 'saurabh',20,"F"),
     (datetime.date(2018,1,12), 'Bala',26,"F"),
     (datetime.date(2018,7,9), 'Jules',19,"F") ,
     (datetime.date(2018,3,18), 'Arild',43,"M"),
     (datetime.date(2018,1,5), 'sarah',20,"M"),
     (datetime.date(2018,8,12), 'Boly',33,"M"),
     (datetime.date(2018,4,6), 'Anita',35,"M"),
     (datetime.date(2018,12,6), 'Jules',22,"M"),
     (datetime.date(2018,7,24), 'Soul',20,"F"),
     (datetime.date(2018,6,17), 'Gral',54,"F"),
     (datetime.date(2018,9,7), 'Apoh',18,"F"),
     (datetime.date(2018,10,4), 'Dony',32,"F"),
     (datetime.date(2018,2,5), 'Tanoh',31,"F"),
     (datetime.date(2018,11,12), 'Issouf',27,"M"),
     (datetime.date(2018,10,3), 'Bilé',29,"M"),
     (datetime.date(2018,5,3), 'Gagnon',20,"M"),
     (datetime.date(2018,3,5), 'Papiss',28,"M"),
     (datetime.date(2018,2,12), 'Kravitz',34,"M"),
     (datetime.date(2018,5,9), 'Mouli',35,"F"),
     (datetime.date(2018,8,3), 'Jacques',27,"F"),
     (datetime.date(2018,12,5), 'soum',22,"F"),
     (datetime.date(2018,4,12), 'MBra',36,"F")]

rdd = spark_session.sparkContext.parallelize(l)
people = rdd.map(lambda x: Row(date=x[0], name=x[1], age=int(x[2]),sex=x[3]))
schemaPeople = spark_session.createDataFrame(people)
schemaPeople.toPandas()

,age,date,name,sex
0,25,2018-01-03,Ankit,F
1,22,2018-02-03,Jalfaizy,F
2,20,2018-01-05,saurabh,F
3,26,2018-01-12,Bala,F
4,19,2018-07-09,Jules,F
5,43,2018-03-18,Arild,M
6,20,2018-01-05,sarah,M
7,33,2018-08-12,Boly,M
8,35,2018-04-06,Anita,M
9,22,2018-12-06,Jules,M


1- compter le nombre de personne totale 
2- compter le nombre de fille et de garcon
3- quel est l'age moyen, median mini et maxi dans chaque groupe (garcon, fille)
4 - quelle est la date de dernière visite de chaque client par rapport à la date d'aujourd'hui (la colonne date correspond à la date de visite)
5 - quels sont les personnes qui ont une date de visite < 400 jours.
6 - verifier s'il existe des valeurs manquantes
7-rajouter une colonne mois correspondant au mois de chaque individu du DataFrame
8-créer une colonne adult qui prend la valeur 0 pour les personnes de moins de 25 ans et 1 sinon

### 1) le nombre de personne totale

In [5]:
print("le nombre de personnes est {} :".format(schemaPeople.count()))

le nombre de personnes est 24 :


### 2) Le nombre de personne par sexe

In [6]:
schemaPeople.groupby("sex").count().show()

+---+-----+
|sex|count|
+---+-----+
|  F|   14|
|  M|   10|
+---+-----+



### 3) L'age moyen, median, minimum et maximum par sexe

In [29]:
tab=schemaPeople.groupby("sex").agg(f.min("age").alias("min_age"),f.max("age").alias("max_age"),f.mean("age").alias("moyenne_age"))
tab.show()

+---+-------+-------+------------------+
|sex|min_age|max_age|       moyenne_age|
+---+-------+-------+------------------+
|  F|     18|     54|27.642857142857142|
|  M|     20|     43|              29.1|
+---+-------+-------+------------------+



In [14]:
medF=schemaPeople.filter(schemaPeople["sex"]=="F").approxQuantile("age",[0.5],0.0)

In [15]:
medM=schemaPeople.filter(schemaPeople["sex"]=="M").approxQuantile("age",[0.5],0.0)

In [17]:
import pandas as pd

In [28]:

med=pd.DataFrame({"median":[medF,medM]})
med

,median
0,[25.0]
1,[28.0]


In [24]:
pd.concat([tab.toPandas(),med],axis=1)

,sex,min_age,max_age,moyenne_age,median
0,F,18,54,27.642857,[25.0]
1,M,20,43,29.100000,[28.0]


### 4) la date de dernière visite de chaque client par rapport à la date d'aujourd'hui

In [50]:
dd = schemaPeople.select("*",
                         f.lit(datetime.date.today()).alias("date_max"))
dd = dd.select("*", f.datediff('date_max', 'date')\
                    .alias('days_since_last_visit'))
dd.show()

+---+----------+--------+---+----------+---------------------+
|age|      date|    name|sex|  date_max|days_since_last_visit|
+---+----------+--------+---+----------+---------------------+
| 25|2018-01-03|   Ankit|  F|2019-11-02|                  668|
| 22|2018-02-03|Jalfaizy|  F|2019-11-02|                  637|
| 20|2018-01-05| saurabh|  F|2019-11-02|                  666|
| 26|2018-01-12|    Bala|  F|2019-11-02|                  659|
| 19|2018-07-09|   Jules|  F|2019-11-02|                  481|
| 43|2018-03-18|   Arild|  M|2019-11-02|                  594|
| 20|2018-01-05|   sarah|  M|2019-11-02|                  666|
| 33|2018-08-12|    Boly|  M|2019-11-02|                  447|
| 35|2018-04-06|   Anita|  M|2019-11-02|                  575|
| 22|2018-12-06|   Jules|  M|2019-11-02|                  331|
| 20|2018-07-24|    Soul|  F|2019-11-02|                  466|
| 54|2018-06-17|    Gral|  F|2019-11-02|                  503|
| 18|2018-09-07|    Apoh|  F|2019-11-02|               

### 5) les personnes qui ont une date de visite < 400 jours

In [49]:
dd.filter(dd["days_since_last_visit"]<400).show()

+---+----------+------+---+----------+---------------------+
|age|      date|  name|sex|  date_max|days_since_last_visit|
+---+----------+------+---+----------+---------------------+
| 22|2018-12-06| Jules|  M|2019-11-02|                  331|
| 32|2018-10-04|  Dony|  F|2019-11-02|                  394|
| 27|2018-11-12|Issouf|  M|2019-11-02|                  355|
| 29|2018-10-03|  Bilé|  M|2019-11-02|                  395|
| 22|2018-12-05|  soum|  F|2019-11-02|                  332|
+---+----------+------+---+----------+---------------------+



In [115]:
dd.select([f.count(f.when(f.isnan(c), c)).alias(c) for c in dd.columns if c not in ['date','date_max']]).show()

+---+----+---+---------------------+
|age|name|sex|days_since_last_visit|
+---+----+---+---------------------+
|  0|   0|  0|                    0|
+---+----+---+---------------------+



### 7) Rajouter une colonne mois dans le dataframe

In [51]:
dd=schemaPeople.select("*", f.month('date').alias("mois"))
dd.show()

+---+----------+--------+---+----+
|age|      date|    name|sex|mois|
+---+----------+--------+---+----+
| 25|2018-01-03|   Ankit|  F|   1|
| 22|2018-02-03|Jalfaizy|  F|   2|
| 20|2018-01-05| saurabh|  F|   1|
| 26|2018-01-12|    Bala|  F|   1|
| 19|2018-07-09|   Jules|  F|   7|
| 43|2018-03-18|   Arild|  M|   3|
| 20|2018-01-05|   sarah|  M|   1|
| 33|2018-08-12|    Boly|  M|   8|
| 35|2018-04-06|   Anita|  M|   4|
| 22|2018-12-06|   Jules|  M|  12|
| 20|2018-07-24|    Soul|  F|   7|
| 54|2018-06-17|    Gral|  F|   6|
| 18|2018-09-07|    Apoh|  F|   9|
| 32|2018-10-04|    Dony|  F|  10|
| 31|2018-02-05|   Tanoh|  F|   2|
| 27|2018-11-12|  Issouf|  M|  11|
| 29|2018-10-03|    Bilé|  M|  10|
| 20|2018-05-03|  Gagnon|  M|   5|
| 28|2018-03-05|  Papiss|  M|   3|
| 34|2018-02-12| Kravitz|  M|   2|
+---+----------+--------+---+----+
only showing top 20 rows



### 8) Colonne adult qui vaut 1 si l'individu est adult 0 sinon

In [52]:
# 8 colonne adult
dd.withColumn("adult",f.when(dd["age"]<25,0).when(dd["age"]>=25,1)).toPandas()

,age,date,name,sex,mois,adult
0,25,2018-01-03,Ankit,F,1,1
1,22,2018-02-03,Jalfaizy,F,2,0
2,20,2018-01-05,saurabh,F,1,0
3,26,2018-01-12,Bala,F,1,1
4,19,2018-07-09,Jules,F,7,0
5,43,2018-03-18,Arild,M,3,1
6,20,2018-01-05,sarah,M,1,0
7,33,2018-08-12,Boly,M,8,1
8,35,2018-04-06,Anita,M,4,1
9,22,2018-12-06,Jules,M,12,0
